In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import pandas as pd
columns="date,time,season,game_id,sr_start,sr_finish,sr_delta,streak_number,result,my_team_sr,enemy_team_sr,team_sr_delta,my_team_1,my_team_2,my_team_3,my_team_4,my_team_5,my_team_6,enemy_team_1,enemy_team_2,enemy_team_3,enemy_team_4,enemy_team_5,enemy_team_6,map,round,capscore,score_distance,time_left,team_role,match_length,charcter_1,character_2,character_3,communication,note,psychological_condition,eliminations,objective_kills,damage,healing,deaths,weapon_accuracy,offensive_assists,defensive_assists,scoped,,,,".split(',')
print(columns)
dataset=pd.read_csv("./Dataset/overwatch-diary.csv",names=columns)


def fit(self, X, y=None):
    return X
def predict(self,X,y=None):
    return X
def test(self):
    return 1



['date', 'time', 'season', 'game_id', 'sr_start', 'sr_finish', 'sr_delta', 'streak_number', 'result', 'my_team_sr', 'enemy_team_sr', 'team_sr_delta', 'my_team_1', 'my_team_2', 'my_team_3', 'my_team_4', 'my_team_5', 'my_team_6', 'enemy_team_1', 'enemy_team_2', 'enemy_team_3', 'enemy_team_4', 'enemy_team_5', 'enemy_team_6', 'map', 'round', 'capscore', 'score_distance', 'time_left', 'team_role', 'match_length', 'charcter_1', 'character_2', 'character_3', 'communication', 'note', 'psychological_condition', 'eliminations', 'objective_kills', 'damage', 'healing', 'deaths', 'weapon_accuracy', 'offensive_assists', 'defensive_assists', 'scoped', '', '', '', '']
